In [1]:
!pip -q install --no-deps "xformers<0.0.27"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 3.8 MB/s eta 0:00:00


In [2]:
!pip -q install accelerate peft bitsandbytes transformers trl safetensors "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from accelerate import init_empty_weights, infer_auto_device_map, disk_offload
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
# Load the dataset
dataset = load_dataset("nvidia/HelpSteer", split='validation').train_test_split(test_size=0.1)

# Prepare the prompt template
def generate_prompt(data_point):
    result = f"[INST] {data_point['prompt']} [/INST] {data_point['response']}"
    return result

# Function to add a 'text' column based on the generated prompt
def add_text(example):
    example["text"] = generate_prompt(example)
    # Optionally remove the original columns to clean up the dataset
    example.pop("prompt", None)
    example.pop("response", None)
    return example

# Apply the function to add the 'text' column
dataset = dataset.map(add_text)

# Display the modified dataset structure and the first example
print(dataset)
print(dataset['train'][0]["text"])
print(dataset['test'][0]["text"])

Generating train split:   0%|          | 0/35331 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1789 [00:00<?, ? examples/s]

Map:   0%|          | 0/1610 [00:00<?, ? examples/s]

Map:   0%|          | 0/179 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['helpfulness', 'correctness', 'coherence', 'complexity', 'verbosity', 'text'],
        num_rows: 1610
    })
    test: Dataset({
        features: ['helpfulness', 'correctness', 'coherence', 'complexity', 'verbosity', 'text'],
        num_rows: 179
    })
})
[INST] Write an article about the top 5 best smart home gadgets that you can buy right now in 2023. [/INST] In 2023, smart home gadgets are becoming more and more popular, with new products being released every day. With so many options available, it can be challenging to determine which ones are worth purchasing. To help you out, we have compiled a list of the top five best smart home gadgets that you can buy right now.

The first gadget on our list is the Amazon Echo Dot. This device is a voice-controlled smart speaker that allows you to control your smart home devices, play music, and answer questions. It has a built-in Alexa assistant that can help you with various tasks, su

In [6]:
# Initialize the model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/Meta-Llama-3-8B-Instruct",
    max_seq_length = 4096,
    dtype = torch.float16,
    load_in_4bit = True,
)

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

In [7]:
infer_auto_device_map(model) # Checking which device the layers are mapped - 0:gpu, cpu, disk

OrderedDict([('', 0)])

In [8]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaR

In [9]:
# LoRA config
model = FastLanguageModel.get_peft_model(
    model = model,
    r = 8, # LoRA rank
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj", "lm_head"],
    lora_alpha = 16,
    lora_dropout = 0.05, # LoRA dropout
    bias = "none", # LoRA bias ("none","all", "lora_only")
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 100,
    use_rslora = False,  # Rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2024.7 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


Unsloth: Casting lm_head to float32


In [10]:
training_arguments = TrainingArguments(
    output_dir="/content/train_logs",
    fp16=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=1,
    eval_accumulation_steps=1,
    evaluation_strategy='epoch',
    optim="paged_lion_32bit",
    learning_rate=1e-3,
    lr_scheduler_type="cosine",
    max_grad_norm=0.3,
    warmup_ratio=0.01,
    weight_decay=0.001,
    save_strategy='epoch',
    group_by_length=False,
    report_to="none"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
### Custom loss
class CustomTrainer(SFTTrainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        inputs_ = inputs.get("input_ids")
        logits_ = model(**inputs).get("logits")
        # Shift so that tokens < n predict n
        shift_labels = inputs_[..., 1:].contiguous()
        shift_logits = logits_[..., :-1, :].contiguous()
        # Calculate per-token loss
        loss_fct = torch.nn.CrossEntropyLoss(reduce=False)
        loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
        # Resize and average loss per sample
        loss_per_sample = loss.view(shift_logits.size(0), shift_logits.size(1)).mean(axis=1)
        # Calculate weighted average
        weighted_loss = (loss_per_sample * 0.75).mean()
        if return_outputs:
            return weighted_loss, {"shift_labels": shift_labels, "shift_logits": shift_logits}
        return weighted_loss

In [12]:
sft_trainer = CustomTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    dataset_text_field="text",
    args=training_arguments,
    max_seq_length=4096,
    packing=False,
    compute_metrics=None
)

Map:   0%|          | 0/1610 [00:00<?, ? examples/s]

Map:   0%|          | 0/179 [00:00<?, ? examples/s]

In [13]:
sft_trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,610 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 1
\        /    Total batch size = 2 | Total steps = 2,415
 "-____-"     Number of trainable parameters = 546,308,096


Epoch,Training Loss,Validation Loss
1,3.408600,2.358793
2,1.979500,1.855194
3,1.781300,1.612248


TrainOutput(global_step=2415, training_loss=2.2008844663884575, metrics={'train_runtime': 8712.4122, 'train_samples_per_second': 0.554, 'train_steps_per_second': 0.277, 'total_flos': 2.1059590228298957e+17, 'train_loss': 2.2008844663884575, 'epoch': 3.0})

In [14]:
sft_trainer.model.push_to_hub('sun000090/Llama3_8B_Instruct_FT')
sft_trainer.tokenizer.push_to_hub('sun000090/Llama3_8B_Instruct_FT')
sft_trainer.model.config.push_to_hub('sun000090/Llama3_8B_Instruct_FT')

README.md:   0%|          | 0.00/594 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/2.19G [00:00<?, ?B/s]

Saved model to https://huggingface.co/sun000090/Llama3_8B_Instruct_FT


CommitInfo(commit_url='https://huggingface.co/sun000090/Llama3_8B_Instruct_FT/commit/4d89617f47643eed141a4c67e6b143b8d7dcc021', commit_message='Upload config', commit_description='', oid='4d89617f47643eed141a4c67e6b143b8d7dcc021', pr_url=None, pr_revision=None, pr_num=None)